In [20]:
!pip install ipywidgets
!pip install pyinstaller

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [18]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider, Layout

liefermenge = float(input("Jährliche Liefermenge [in kWh]: "))
deckungsbeitragzielvorgabe = float(input("Zielvorgabe [in CHF/Jahr]: "))

def plot_graph(grundpreis_slider, strukturierungszuschlag_slider):
    liefermenge_mwh = liefermenge / 1000

    if liefermenge_mwh < 500:
        segment = "C-Kunden"
        segment_liefermenge = np.linspace(100, 500, 10)
    elif liefermenge_mwh < 4000:
        segment = "B-Kunden"
        segment_liefermenge = np.linspace(500, 4000, 10)
    else:
        segment = "A-Kunden"
        segment_liefermenge = np.linspace(4000, 30000, 10)

    grundpreis = grundpreis_slider * 12
    strukturierungszuschlag_mwh = strukturierungszuschlag_slider * 1000
    strukturierungszuschlag_menge = segment_liefermenge * strukturierungszuschlag_mwh / 100

    def check_marge(deckungsbeitragzielvorgabe, liefermenge_mwh, grundpreis, strukturierungszuschlag_mwh, segment_liefermenge):
        differenz = (deckungsbeitragzielvorgabe - grundpreis - strukturierungszuschlag_mwh * liefermenge_mwh / 100)
        differenz_marge = round((differenz / liefermenge_mwh * 100) / 1000, 3)
        if differenz_marge < 0:
            differenz_marge = 0
            differenz_marge_menge_mwh = segment_liefermenge * 0
            return differenz_marge, differenz_marge_menge_mwh
        #print(f"Notwendiger Aufschlag auf Arbeitspreis von {differenz_marge} Rappen/kWh")
        differenz_marge_menge_mwh = (segment_liefermenge * differenz_marge * 1000) / 100
        return differenz_marge, differenz_marge_menge_mwh

    differenz_marge, differenz_marge_menge_mwh = check_marge(deckungsbeitragzielvorgabe, liefermenge_mwh, grundpreis,
                                                              strukturierungszuschlag_mwh, segment_liefermenge)

    def check_beitrag(grundpreis, differenz_marge, strukturierungszuschlag_mwh, segment_liefermenge):
        beitrag_marge_menge_mwh = grundpreis + segment_liefermenge * (
                differenz_marge * 1000 + strukturierungszuschlag_mwh) / 100
        return beitrag_marge_menge_mwh

    beitrag_marge_menge = check_beitrag(grundpreis, differenz_marge, strukturierungszuschlag_mwh, segment_liefermenge)
    
    print(f"Um die jährliche Deckungszielvorgabe von CHF {deckungsbeitragzielvorgabe} für die jährliche Liefermenge {liefermenge_mwh * 1000} kWh zu erreichen,\n"
          f"braucht es bei einem monatlichen Grundpreis von CHF {grundpreis} und einem Strukturierungszuschlag von {strukturierungszuschlag_mwh / 1000} Rp./kWh\n"
          f"einen zusätzlichen Margenaufschlag auf dem Arbeitspreis von {differenz_marge} Rp./kWh.")
    
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [2, 1]}, figsize=(14, 12))

    ax1.axhline(y=grundpreis, color='g', linestyle='--', label='Grundpreis')
    ax1.axhline(y=deckungsbeitragzielvorgabe, color='r', linestyle='--', label='Deckungsbeitragzielvorgabe')

    ax1.axvline(x=liefermenge_mwh, color='b', linestyle='--', label='Eingegebene Liefermenge')

    ax1.plot(segment_liefermenge, strukturierungszuschlag_menge, label='Strukturierungszuschlag')
    ax1.plot(segment_liefermenge, differenz_marge_menge_mwh, label=f'Differenzmarge für {segment}')

    ax1.set_ylabel('[CHF]')
    ax1.set_xlabel('Jährliche Liefermenge [MWh]')
    ax1.legend()
    ax1.set_title(f"Marge für {segment}")

    ax2.axhline(y=deckungsbeitragzielvorgabe, color='r', linestyle='--', label='Deckungsbeitragzielvorgabe')
    ax2.fill_between(segment_liefermenge, beitrag_marge_menge, alpha=0.3, color='orange')

    ax2.axvline(x=liefermenge_mwh, color='b', linestyle='--', label='Eingegebene Liefermenge')
    ax2.plot(segment_liefermenge, beitrag_marge_menge, label='Margenentwicklung', color='orange')

    ax2.set_xlabel('Jährliche Liefermenge [MWh]')
    ax2.set_ylabel('[CHF]')
    ax2.legend()

    plt.show()

grundpreis_slider = FloatSlider(value=0, min=0, max=500, step=10, description='Grundpreis [CHF/Monat]', style={'description_width':'initial'} , layout=Layout(width='80%'))
strukturierungszuschlag_slider = FloatSlider(value=0, min=0, max=3, step=0.05,
                                             description='Strukturierungszuschlag [Rp./kWh]', style={'description_width':'initial'}, layout=Layout(width='80%'))

interact(plot_graph, grundpreis_slider=grundpreis_slider, strukturierungszuschlag_slider=strukturierungszuschlag_slider)


Jährliche Liefermenge [in kWh]: 350000
Zielvorgabe [in CHF/Jahr]: 5000


interactive(children=(FloatSlider(value=0.0, description='Grundpreis [CHF/Monat]', layout=Layout(width='80%'),…

<function __main__.plot_graph(grundpreis_slider, strukturierungszuschlag_slider)>